In [1]:
import src.login_config as config
import src.helpers as helpers
import src.data_scrape as ds
import pandas as pd

In [5]:
# Retrieve of all sold listing links that was previously scraped in 1-GVA_data_scrape_listing_links notebook
combined_df = helpers.read_csv_files('sold_listings')

print(len(combined_df))

17529


In [9]:
richmond_df = pd.read_csv('richmond_sold_listings_new.csv',index_col = [0])

The code will take some time to run. Therefore, I have split the code to run in batches (by cities). 

In [ ]:
# sample. split them by batch (city). With the exception of Vancouver, I have split it into 2 batches due to the immense size.
# To replicate this, rename the dataframe and change the name of the "City. For example, if I want Burnaby then change dataframe to the folllowing:
# burnaby_df = combined_df[combined_df['City'] == 'burnaby'].reset_index(drop = True) "

vancouver_df = combined_df[combined_df['City'] == 'vancouver'].reset_index(drop = True)

van_df1 = vancouver_df[0:2700]
van_df2 = vancouver_df[2700:].reset_index(drop = True)

In [ ]:
import time
import pandas as pd

batch = richmond_df

browser = ds.selenium_start()
ds.first_time_login(browser)

df_list = list()
for i in range(0,len(batch)):
    start_time = time.time()
    
    table = ds.retrieve_sold_listing_description(browser, batch['Listing link'][i])
    df_list.append(table)
    print(i)
    end_time = time.time()
    
    execution_time = end_time - start_time
    print("Execution time:", execution_time, "seconds")

full_table = pd.concat(df_list, axis = 0, ignore_index = True)

In [ ]:
# Save progress in csv file
# For other cities they are saved in this format: burnaby_complete, coquitlam_complete, etc..)
full_table.to_csv('richmond2_complete.csv')

In [2]:
# Read in all the csv files that was previously run in the code above for all cities
cities_df = helpers.read_csv_files('_complete')

# some listings have none values. let's remove it
cities_df = cities_df[~pd.isnull(cities_df['list_date'])].reset_index(drop = True)
cities_df['City'] = cities_df['City'].replace({'vancouver1': 'vancouver', 'vancouver2':'vancouver'})

In [5]:
#Store this in a database

import pandas as pd
## connect to database
import sqlite3

conn = sqlite3.connect("housing_information.db")

##push the dataframe to sql 
cities_df.to_sql("df_listings", conn, if_exists="replace", index = False)

##create the table

conn.execute(
    """
    create table my_table as 
    select * from df_listings
    """)

In [4]:
# Reading data in from database
pd.read_sql_query('SELECT * FROM df_listings', conn)

,house_type,size,maint_fee,approx_age,mls_number,levels,garage,garage_size,taxes,avg_price_sqft,property_addr,bedroom,bathroom,list_date,list_price,end_date,sold_price,City
0,Apartment/Condominium,959 sqft,None,2021.0,R2834693,Residential,Yes,1.0,"$2,944","$1,157","Property Address: Juneau Street, Burnaby, Brit...",2.0,2.0,04/03/2024,"$995,000",12/03/2024,"$970,000",burnaby
1,Apartment/Condominium,733 sqft,None,1973.0,R2821009,Residential,Yes,1.0,"$1,044",$600,"Property Address: 105 - 7180 Linden Avenue, Bu...",1.0,1.0,30/09/2023,"$440,000",12/03/2024,"$435,000",burnaby
2,Apartment/Condominium,1205 sqft,None,1975.0,R2773433,Residential,Yes,1.0,"$1,404",$662,"Property Address: 703 - 3737 Bartlett Court, B...",2.0,1.0,11/12/2023,"$748,000",12/03/2024,"$738,000",burnaby
3,Single Family Residence,2513 sqft,None,1969.0,R2820239,Residential,Yes,5.0,"$5,066",$756,"Property Address: 1035 Holdom Avenue, Burnaby,...",5.0,3.0,04/03/2024,"$1,850,000",08/03/2024,"$1,800,000",burnaby
4,Apartment/Condominium,835 sqft,None,2017.0,R2837353,Residential,Yes,2.0,"$2,696","$1,154","Property Address: 4304 - 4485 Skyline Drive, B...",2.0,2.0,10/12/2023,"$963,800",04/03/2024,"$928,228",burnaby
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14911,Apartment/Condominium,514 sqft,None,2010.0,R2592520,Residential,No,0.0,"$1,635","$1,165","Property Address: 710 - 535 Smithe Street, Van...",1.0,1.0,13/06/2021,"$599,000",25/06/2021,"$595,000",vancouver
14912,Townhouse,1103 sqft,None,2018.0,R2592341,Residential,Yes,0.0,"$2,765",$905,"Property Address: 365 E 16, Vancouver, British...",2.0,2.0,13/06/2021,"$999,000",25/06/2021,"$990,000",vancouver
14913,Multi Family,1537 sqft,None,2004.0,R2550233,Residential,Yes,0.0,"$5,405","$1,098","Property Address: 8469 French, Vancouver, Brit...",6.0,3.0,11/03/2021,"$1,688,800",25/06/2021,"$1,628,900",vancouver
14914,Townhouse,2208 sqft,None,2020.0,R2594052,Residential,Yes,0.0,None,$797,"Property Address: 528 E 2nd, North Vancouver, ...",4.0,3.0,15/06/2021,"$1,759,900",16/06/2021,"$1,759,900",vancouver
